In [88]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from google.colab import drive
import os
drive.mount('/content/drive/', force_remount = True)
ROOT_PATH="/content/drive/My Drive/IFT6285-devoir6/"
IMAGES_PATH = ROOT_PATH
os.chdir(ROOT_PATH)
os.getcwd()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Mounted at /content/drive/


'/content/drive/My Drive/IFT6285-devoir6'

In [89]:
list_of_cases = os.listdir("./data")
# list_of_cases =  ['blog-50-50-10', 'blog-50-100-10', 'blog-100-10-10', 'blog-100-50-10', 'blog-100-100-10', 'blog-quiz']

In [50]:
from sklearn.externals import joblib
# train_set_name = "C:/Users/Euge/Documents/Session1-Maitrise/devoir6/blog-10-10-10/train.csv"
# testopen_set_name = "C:/Users/Euge/Documents/Session1-Maitrise/devoir6/blog-10-10-10/test-open.csv"
# testclose_set_name = "C:/Users/Euge/Documents/Session1-Maitrise/devoir6/blog-10-10-10/test-closed.csv"
train_set_name = "./data/blog-10-10-10/train.csv"
testopen_set_name = "./data/blog-10-10-10/test-open.csv"
testclose_set_name = "./data/blog-10-10-10/test-closed.csv"

result_folder = './result/'
result_folder = './result/bkp/'

train_accuracy_score = {}
testopen_accuracy_score = {}
# train_accuracy_score = joblib.load(result_folder + 'train_accuracy_score.pkl', mmap_mode=None)
# testopen_accuracy_score = joblib.load(result_folder + 'testopen_accuracy_score.pkl', mmap_mode=None)

print(train_accuracy_score)
print(testopen_accuracy_score)

{}
{}


# Les paramètres

In [94]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

models = []
params = {}
vectorizer = TfidfVectorizer

#------------------------------------------------------------------
#----------------------------MultinomialNB-------------------------
#------------------------------------------------------------------

model = ComplementNB
models.append(model)
param = {
'vect__max_features': [1000,2000,3000,4000],
'model__alpha':  [0.1, 0.01, 1],
    }
params[model.__name__] = param

#------------------------------------------------------------------
#----------------------------MLP-----------------------------------
#------------------------------------------------------------------
# model = MLPClassifier
# models.append(model)
# param = {
# 'vect__max_features': [1000, 4000],
# 'model__hidden_layer_sizes':  [(50,50)],
# 'model__solver':  ['adam'],
# 'model__learning_rate' : ['constant'], 
# 'model__learning_rate_init' :  [0.001, 0.01,0.1],
# 'model__max_iter' : [2000],
#     }
# params[model.__name__] = param

#------------------------------------------------------------------
#----------------------------Xgboost-------------------------------
#------------------------------------------------------------------

# model = XGBClassifier
# models.append(model)
# # param = {
# #         'vect__max_features': [1000, 4000],
# #         'model__learning_rate'    : [0.01, 0.3],
# #         'model__min_child_weight': [1, 5],
# #         'model__subsample': [0.6, 1.0],
# #         'model__colsample_bytree': [0.3, 1.0],
# #         'model__max_depth': [3, 6, 10],
# #         'model__objective': ['multi:softmax'],
# #         'model__n_estimators': [100],
# #         'model__eval_metric': ['logloss'],
# # }
# param = {
#         'vect__max_features': [4000],
#         'model__learning_rate'    : [0.01],
#         'model__min_child_weight': [1],
#         'model__subsample': [0.6],
#         'model__colsample_bytree': [0.3],
#         'model__max_depth': [10],
#         'model__objective': ['multi:softmax'],
#         'model__n_estimators': [100],
#         'model__eval_metric': ['logloss'],
# }
# params[model.__name__] = param


#------------------------------------------------------------------
#----------------------------Logistic Regression-------------------
#------------------------------------------------------------------

# model = LogisticRegression
# models.append(model)
# param = {
#         'vect__max_features': [1000, 4000],
#         'model__max_iter': [1000],
# }

# params[model.__name__] = param


print(params)

{'ComplementNB': {'vect__max_features': [1000, 2000, 3000, 4000], 'model__alpha': [0.1, 0.01, 1]}}


# GridSearch

In [56]:
from train_model import get_best_model, load_data, print_scores
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.externals import joblib
# import logging

result_folder = './result/'
# train_accuracy_score = {}
# testopen_accuracy_score = {}
train_accuracy_score = joblib.load(result_folder + 'train_accuracy_score.pkl', mmap_mode=None)
testopen_accuracy_score = joblib.load(result_folder + 'testopen_accuracy_score.pkl', mmap_mode=None)
for case in list_of_cases:
    print('-------------------------- Case:' + case + ' --------------------------------')
    train_set_name = "./data/"  + case + "/train.csv"
    testopen_set_name = "./data/" + case + "/test-open.csv"
    testclose_set_name = "./data/" + case + "/test-closed.csv"
    
    i=0
    X_train, y_train, X_test, y_test = load_data(train_set_name, testopen_set_name, testclose_set_name)

    for i, model  in enumerate(models):
        # model = models[i]
        best_params_, gs, model_prediction, train_score, test_score = get_best_model(model,params[model.__name__],X_train, y_train, X_test, y_test)
        best_params_
        # print(classification_report(y_test, model_prediction))
        # plot_confusion_matrix(gs, X_test, y_test, cmap='Blues', values_format='d')
        train_accuracy_score[model.__name__ + '_' + case] = accuracy_score(y_train, gs.predict(X_train))
        testopen_accuracy_score[model.__name__ + '_' + case] = accuracy_score(y_test, gs.predict(X_test))
        # testclose_accuracy_score[case + '_' + model] = accuracy_score(y_test, gs.predict(X_test))
        joblib.dump(gs, result_folder + model.__name__ + '_' + case +'_gs.pkl')
        joblib.dump(train_accuracy_score, result_folder + 'train_accuracy_score.pkl')
        joblib.dump(testopen_accuracy_score, result_folder + 'testopen_accuracy_score.pkl')


        print(train_accuracy_score)
        print(testopen_accuracy_score)


-------------------------- Case:blog-10-10-10 --------------------------------
Now loading data and spliting
./data/blog-10-10-10/train.csv


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Now training the model : ComplementNB
Fitting 10 folds for each of 32 candidates, totalling 320 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1906s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  22 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1666s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  80 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 116 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 160 tasks    

{'model__alpha': 0.1, 'vect__max_features': 2000, 'vect__use_idf': True}

...

{'model__hidden_layer_sizes': (50, 50),
 'model__learning_rate': 'constant',
 'model__learning_rate_init': 0.001,
 'model__max_iter': 2000,
 'model__solver': 'adam',
 'vect__max_features': 4000,
 'vect__use_idf': True}

['./result/MLPClassifier_blog-quiz_gs.pkl']

['./result/train_accuracy_score.pkl']

['./result/testopen_accuracy_score.pkl']

{'ComplementNB_blog-10-10-10': 0.9797979797979798, 'MLPClassifier_blog-10-10-10': 0.98989898989899, 'ComplementNB_blog-10-50-10': 0.9879759519038076, 'MLPClassifier_blog-10-50-10': 1.0, 'ComplementNB_blog-10-100-10': 0.97997997997998, 'MLPClassifier_blog-10-100-10': 0.998998998998999, 'ComplementNB_blog-50-10-10': 0.9859719438877755, 'MLPClassifier_blog-50-10-10': 0.9959919839679359, 'ComplementNB_blog-50-50-10': 0.9319727891156463, 'MLPClassifier_blog-50-50-10': 0.9959983993597439, 'ComplementNB_blog-50-100-10': 0.8767753550710142, 'MLPClassifier_blog-50-100-10': 0.9975995199039808, 'ComplementNB_blog-100-10-10': 0.984984984984985, 'MLPClassifier_blog-100-10-10': 0.997997997997998, 'ComplementNB_blog-100-50-10': 0.865373074614923, 'MLPClassifier_blog-100-50-10': 0.992998599719944, 'ComplementNB_blog-100-100-10': 0.7533753375337534, 'MLPClassifier_blog-100-100-10': 0.9861986198619862, 'ComplementNB_blog-quiz': 0.985985985985986, 'MLPClassifier_blog-quiz': 0.996996996996997}
{'Complemen

# GridSearch pour un modele en particulier

In [ ]:
from train_model import get_best_model, load_data, print_scores
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.externals import joblib

from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline


# model= ComplementNB
# param = {
# # 'vect__use_idf': [True],
# # 'vect__max_features': [1000],
# 'model__alpha':  [0.009],
#     }
model = MLPClassifier
param = {
'vect__max_features': [4000],
'model__hidden_layer_sizes':  [(200,200)],
'model__solver':  ['adam'],
'model__learning_rate' : ['constant'], 
'model__learning_rate_init' :  [0.001, 0.01,0.1],
'model__max_iter' : [2000],
    }


case = 'blog-10-10-10'

train_set_name = "./data/"  + case + "/train.csv"
testopen_set_name = "./data/" + case + "/test-open.csv"
testclose_set_name = "./data/" + case + "/test-closed.csv"
    
X_train, y_train, X_test, y_test = load_data(train_set_name, testopen_set_name, testclose_set_name)

pipe = Pipeline([
        ('vect', vectorizer()),
        # ('scaler', MaxAbsScaler()),
        # ('scaler', StandardScaler()),
        ('model', model())
    ])
gs = GridSearchCV(pipe,  # what object are we optimizing?
                      param_grid=param,  # what parameters values are we searching?
                      cv=10,
                      scoring='accuracy', verbose=10,
                      n_jobs=-1)  # 5-fold cross-validation.
# gs=model(alpha=0.009)
# gs.fit(X_train, y_train)

train_accuracy_s= accuracy_score(y_train, gs.predict(X_train))
testopen_accuracy_s= accuracy_score(y_test, gs.predict(X_test))
print(train_accuracy_s)
print(testopen_accuracy_s)

# Estimation des modele avec le meiulleurs parametres du gridsearch

In [103]:
from train_model_vect import get_best_model, load_data, print_scores
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.externals import joblib
import re
# import logging

result_folder = './result/'

list_of_cases = ['blog-50-10-10']
train_accuracy_score = {}
testopen_accuracy_score = {}
for case in list_of_cases:
    print('-------------------------- Case:' + case + ' --------------------------------')
    train_set_name = "./data/"  + case + "/train.csv"
    testopen_set_name = "./data/" + case + "/test-open.csv"
    testclose_set_name = "./data/" + case + "/test-closed.csv"
    
    i=0
    X_train, y_train, X_test, y_test = load_data(train_set_name, testopen_set_name, testclose_set_name)

    for i, model  in enumerate(models):
        model_param = {}
        vect_param = {}
        gs_reload = joblib.load(result_folder + model.__name__ + '_' + case +'_gs.pkl', mmap_mode=None)
        # print(gs_reload.best_params_)

        for key, value in gs_reload.best_params_.items():
            if "model" in key:
                new_key = re.sub('model__','',key)
                # value =  (200, 200,200) if new_key =='hidden_layer_sizes' else value
                model_param[new_key] = value
            if "vect" in key:
                new_key = re.sub('vect__','',key)
                vect_param[new_key] = value
        print(model.__name__)
        print(model_param)
        print(vect_param)

        m=model(**model_param)
        m.fit(X_train, y_train)
        m

        train_accuracy_score[model.__name__ + '_' + case] = accuracy_score(y_train, m.predict(X_train))
        testopen_accuracy_score[model.__name__ + '_' + case] = accuracy_score(y_test, m.predict(X_test))

        joblib.dump(m, './final_result/' + model.__name__ + '_' + case +'_m.pkl')
        joblib.dump(train_accuracy_score,'./final_result/' + 'train_accuracy_model.pkl')
        joblib.dump(testopen_accuracy_score,'./final_result/' + 'testopen_accuracy_model.pkl')


        print(train_accuracy_score)
        print(testopen_accuracy_score)





-------------------------- Case:blog-50-10-10 --------------------------------


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


ComplementNB
{'alpha': 0.1}
{'max_features': 4000, 'use_idf': False}


ComplementNB(alpha=0.1, class_prior=None, fit_prior=True, norm=False)

ComplementNB(alpha=0.1, class_prior=None, fit_prior=True, norm=False)

['./final_result/ComplementNB_blog-50-10-10_m.pkl']

['./final_result/train_accuracy_model.pkl']

['./final_result/testopen_accuracy_model.pkl']

{'ComplementNB_blog-50-10-10': 1.0}
{'ComplementNB_blog-50-10-10': 0.3687374749498998}


# Novelty and Outlier Detection

In [ ]:
case
from sklearn.neighbors import LocalOutlierFactor

lof = LocalOutlierFactor(novelty=True, n_neighbors = 10, contamination  = 0.0005)
lof.fit(X_train)
# lof.decision_function(X_test)
lof.predict(X_test)
# lof.score_samples(X_test)